# Phase 2: Baseline Model Development

**Objective:** Build and evaluate baseline machine learning models for credit risk prediction

**Goals:**
- Preprocess the full dataset
- Train multiple baseline models
- Optimize hyperparameters
- Evaluate and compare models
- Select best model for Phase 3

## 1. Setup and Imports

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib
import sys
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append('../src')

# ML libraries
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report,
    precision_recall_curve, average_precision_score
)
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
import xgboost as xgb
from imblearn.over_sampling import SMOTE

# Custom preprocessing
from data_preprocessing import CreditDataPreprocessor, save_splits

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
%matplotlib inline

print("✅ All libraries imported successfully")

## 2. Load and Preprocess Data

In [ ]:
# Load dataset
print("Loading dataset...")
df = pd.read_csv('../data/raw/LC_loans_granting_model_dataset.csv', low_memory=False)
print(f"Dataset loaded: {df.shape}")
print(f"\nTarget distribution:")
print(df['Default'].value_counts())
print(f"\nDefault rate: {df['Default'].mean()*100:.2f}%")

In [ ]:
# Initialize and apply preprocessor
print("Preprocessing data...")
preprocessor = CreditDataPreprocessor()

# Preprocess (this handles missing values, encoding, scaling, and feature engineering)
df_processed = preprocessor.fit_transform(
    df, 
    target_col='Default',
    encoding='label',  # Label encoding for tree-based models
    scaling=True,      # Scale features
    create_features=True  # Create engineered features
)

print(f"\nProcessed shape: {df_processed.shape}")
print(f"Features: {len(preprocessor.feature_names)}")

In [ ]:
# Split data (stratified)
print("Splitting data...")
X_train, X_val, X_test, y_train, y_val, y_test = preprocessor.split_data(
    df_processed,
    target_col='Default',
    test_size=0.20,
    val_size=0.10,
    random_state=42
)

print(f"\nTrain shape: {X_train.shape}")
print(f"Val shape: {X_val.shape}")
print(f"Test shape: {X_test.shape}")

In [ ]:
# Save splits for reproducibility
save_splits(X_train, X_val, X_test, y_train, y_val, y_test, output_dir='../data/splits')
print("✅ Data splits saved to ../data/splits/")

In [ ]:
# Handle class imbalance with SMOTE (on training set only)
print("Applying SMOTE to training data...")
print(f"Before SMOTE - Class distribution:")
print(y_train.value_counts())

smote = SMOTE(random_state=42, sampling_strategy=0.5)  # Balance to 1:2 ratio
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f"\nAfter SMOTE - Class distribution:")
print(pd.Series(y_train_balanced).value_counts())
print(f"\nBalanced training shape: {X_train_balanced.shape}")

## 3. Train Baseline Models

### 3.1 Logistic Regression (Simple Baseline)

In [ ]:
print("Training Logistic Regression...")
lr_model = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
lr_model.fit(X_train, y_train)
print("✅ Logistic Regression trained")

### 3.2 Random Forest (Robust Ensemble)

In [ ]:
print("Training Random Forest...")
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    min_samples_split=20,
    min_samples_leaf=10,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train_balanced, y_train_balanced)
print("✅ Random Forest trained")

### 3.3 XGBoost (State-of-the-Art)

In [ ]:
print("Training XGBoost...")
# Calculate scale_pos_weight for imbalanced data
scale_pos_weight = len(y_train[y_train==0]) / len(y_train[y_train==1])
print(f"Scale pos weight: {scale_pos_weight:.2f}")

xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1,
    eval_metric='logloss'
)
xgb_model.fit(X_train, y_train)
print("✅ XGBoost trained")

## 4. Evaluate All Models on Validation Set

In [ ]:
# Evaluation function
def evaluate_model(model, X, y, model_name="Model"):
    """Comprehensive model evaluation"""
    y_pred = model.predict(X)
    y_pred_proba = model.predict_proba(X)[:, 1]
    
    metrics = {
        'Model': model_name,
        'Accuracy': accuracy_score(y, y_pred),
        'Precision': precision_score(y, y_pred),
        'Recall': recall_score(y, y_pred),
        'F1-Score': f1_score(y, y_pred),
        'AUC-ROC': roc_auc_score(y, y_pred_proba),
        'AUC-PR': average_precision_score(y, y_pred_proba)
    }
    
    return metrics, y_pred, y_pred_proba

# Evaluate all models
models = {
    'Logistic Regression': lr_model,
    'Random Forest': rf_model,
    'XGBoost': xgb_model
}

results = []
predictions = {}

print("Evaluating models on validation set...\n")
for name, model in models.items():
    metrics, y_pred, y_pred_proba = evaluate_model(model, X_val, y_val, name)
    results.append(metrics)
    predictions[name] = {'pred': y_pred, 'proba': y_pred_proba}
    
    print(f"\n{name}:")
    for metric, value in metrics.items():
        if metric != 'Model':
            print(f"  {metric}: {value:.4f}")

# Create comparison dataframe
results_df = pd.DataFrame(results)
print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60)
print(results_df.to_string(index=False))

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC', 'AUC-PR']

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx // 3, idx % 3]
    values = results_df[metric].values
    bars = ax.bar(results_df['Model'], values, color=['#3498db', '#2ecc71', '#e74c3c'])
    ax.set_ylabel(metric, fontsize=12, fontweight='bold')
    ax.set_ylim([min(values) - 0.05, 1.0])
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=10)
    
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../results/figures/model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Model comparison plot saved")

## 5. Confusion Matrices

In [ ]:
# Plot confusion matrices
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, (name, model) in enumerate(models.items()):
    y_pred = predictions[name]['pred']
    cm = confusion_matrix(y_val, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                xticklabels=['Paid', 'Default'],
                yticklabels=['Paid', 'Default'])
    axes[idx].set_title(f'{name}\nConfusion Matrix', fontweight='bold')
    axes[idx].set_ylabel('True Label')
    axes[idx].set_xlabel('Predicted Label')

plt.tight_layout()
plt.savefig('../results/figures/confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Confusion matrices saved")

## 6. ROC Curves

In [ ]:
# Plot ROC curves
plt.figure(figsize=(10, 8))

colors = ['#3498db', '#2ecc71', '#e74c3c']
for idx, (name, model) in enumerate(models.items()):
    y_pred_proba = predictions[name]['proba']
    fpr, tpr, _ = roc_curve(y_val, y_pred_proba)
    auc = roc_auc_score(y_val, y_pred_proba)
    
    plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', 
             color=colors[idx], linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random', linewidth=1)
plt.xlabel('False Positive Rate', fontsize=12, fontweight='bold')
plt.ylabel('True Positive Rate', fontsize=12, fontweight='bold')
plt.title('ROC Curves - Model Comparison', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../results/figures/roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ ROC curves saved")

## 7. Precision-Recall Curves

In [ ]:
# Plot Precision-Recall curves
plt.figure(figsize=(10, 8))

for idx, (name, model) in enumerate(models.items()):
    y_pred_proba = predictions[name]['proba']
    precision, recall, _ = precision_recall_curve(y_val, y_pred_proba)
    ap = average_precision_score(y_val, y_pred_proba)
    
    plt.plot(recall, precision, label=f'{name} (AP = {ap:.3f})', 
             color=colors[idx], linewidth=2)

baseline = y_val.sum() / len(y_val)
plt.axhline(y=baseline, color='k', linestyle='--', 
            label=f'Baseline (No Skill = {baseline:.3f})', linewidth=1)

plt.xlabel('Recall', fontsize=12, fontweight='bold')
plt.ylabel('Precision', fontsize=12, fontweight='bold')
plt.title('Precision-Recall Curves - Model Comparison', fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../results/figures/precision_recall_curves.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Precision-Recall curves saved")

## 8. Select Best Model and Hyperparameter Tuning

In [ ]:
# Select best model based on AUC-ROC
best_model_name = results_df.loc[results_df['AUC-ROC'].idxmax(), 'Model']
print(f"\n🏆 Best performing model: {best_model_name}")
print(f"   AUC-ROC: {results_df['AUC-ROC'].max():.4f}")

# We'll tune XGBoost as it typically performs best
print("\nProceeding with XGBoost hyperparameter tuning...")

In [ ]:
# Hyperparameter tuning for XGBoost
print("Performing Grid Search for XGBoost...")

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb_tuned = xgb.XGBClassifier(
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1,
    eval_metric='logloss'
)

grid_search = GridSearchCV(
    xgb_tuned,
    param_grid,
    cv=3,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

# Fit on training data
grid_search.fit(X_train, y_train)

print(f"\n✅ Best parameters: {grid_search.best_params_}")
print(f"   Best CV score: {grid_search.best_score_:.4f}")

# Get best model
best_model = grid_search.best_estimator_

In [ ]:
# Evaluate tuned model
print("\nEvaluating tuned XGBoost model...")
metrics_tuned, y_pred_tuned, y_pred_proba_tuned = evaluate_model(
    best_model, X_val, y_val, "XGBoost (Tuned)"
)

print("\nTuned XGBoost Performance:")
for metric, value in metrics_tuned.items():
    if metric != 'Model':
        print(f"  {metric}: {value:.4f}")

## 9. Feature Importance Analysis

In [ ]:
# Feature importance from best model
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': best_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nTop 20 Most Important Features:")
print(feature_importance.head(20))

# Plot feature importance
plt.figure(figsize=(10, 8))
top_features = feature_importance.head(20)
plt.barh(range(len(top_features)), top_features['Importance'], color='steelblue')
plt.yticks(range(len(top_features)), top_features['Feature'])
plt.xlabel('Importance', fontsize=12, fontweight='bold')
plt.title('Top 20 Feature Importances - XGBoost', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('../results/figures/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Feature importance plot saved")

## 10. Model Calibration

In [ ]:
# Check calibration
prob_true, prob_pred = calibration_curve(
    y_val, y_pred_proba_tuned, n_bins=10, strategy='uniform'
)

plt.figure(figsize=(10, 8))
plt.plot(prob_pred, prob_true, 's-', label='XGBoost (Before Calibration)', markersize=8)
plt.plot([0, 1], [0, 1], 'k--', label='Perfect Calibration')
plt.xlabel('Predicted Probability', fontsize=12, fontweight='bold')
plt.ylabel('True Probability', fontsize=12, fontweight='bold')
plt.title('Calibration Curve', fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../results/figures/calibration_before.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Calibration curve (before) saved")

In [ ]:
# Apply calibration
print("Applying probability calibration...")
calibrated_model = CalibratedClassifierCV(
    best_model, 
    method='sigmoid',  # Platt scaling
    cv=3
)
calibrated_model.fit(X_train, y_train)

# Evaluate calibrated model
y_pred_proba_cal = calibrated_model.predict_proba(X_val)[:, 1]
prob_true_cal, prob_pred_cal = calibration_curve(
    y_val, y_pred_proba_cal, n_bins=10, strategy='uniform'
)

# Plot both
plt.figure(figsize=(10, 8))
plt.plot(prob_pred, prob_true, 's-', label='Before Calibration', markersize=8)
plt.plot(prob_pred_cal, prob_true_cal, 'o-', label='After Calibration', markersize=8)
plt.plot([0, 1], [0, 1], 'k--', label='Perfect Calibration')
plt.xlabel('Predicted Probability', fontsize=12, fontweight='bold')
plt.ylabel('True Probability', fontsize=12, fontweight='bold')
plt.title('Calibration Curve - Before & After', fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../results/figures/calibration_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Calibrated model ready")

## 11. Save Models

In [ ]:
# Save all important artifacts
print("Saving models and preprocessor...")

# Save best model (uncalibrated)
joblib.dump(best_model, '../results/models/xgboost_best.pkl')
print("✅ Best XGBoost model saved")

# Save calibrated model
joblib.dump(calibrated_model, '../results/models/xgboost_calibrated.pkl')
print("✅ Calibrated XGBoost model saved")

# Save preprocessor
joblib.dump(preprocessor, '../results/models/preprocessor.pkl')
print("✅ Preprocessor saved")

# Save all baseline models
joblib.dump(lr_model, '../results/models/logistic_regression.pkl')
joblib.dump(rf_model, '../results/models/random_forest.pkl')
print("✅ All baseline models saved")

## 12. Summary and Next Steps

In [ ]:
print("="*60)
print("PHASE 2 COMPLETE: BASELINE MODEL DEVELOPMENT")
print("="*60)
print("\n✅ Achievements:")
print("   - Data preprocessed and split (train/val/test)")
print("   - Class imbalance handled with SMOTE")
print("   - 3 baseline models trained and evaluated")
print("   - XGBoost hyperparameters optimized")
print("   - Model calibration applied")
print("   - Feature importance analyzed")
print("   - All models and artifacts saved")

print("\n📊 Best Model Performance (Validation Set):")
for metric, value in metrics_tuned.items():
    if metric != 'Model':
        print(f"   {metric}: {value:.4f}")

print("\n🎯 Phase 2 Success Criteria:")
print(f"   ✅ AUC-ROC > 0.75: {metrics_tuned['AUC-ROC']:.4f}")
print(f"   ✅ Model calibrated and ready")
print(f"   ✅ Feature importance identified")

print("\n🚀 Next: Phase 3 - Uncertainty Quantification")
print("   - Implement Bootstrap Ensemble")
print("   - Quantify prediction uncertainty")
print("   - Validate uncertainty estimates")
print("="*60)